In [1]:
import torch

In [2]:
torch.__version__

'1.0.0'

In [3]:
# CUDA_VISIBLE_DEVICES=2
# torch.cuda.set_device(1)

In [4]:
import os

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [6]:
'''MobileNetV3 in PyTorch.
See the paper "Inverted Residuals and Linear Bottlenecks:
Mobile Networks for Classification, Detection and Segmentation" for more details.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

In [7]:
def Hswish(x,inplace=True):
    return x * F.relu6(x + 3., inplace=inplace) / 6.

def Hsigmoid(x,inplace=True):
    return F.relu6(x + 3., inplace=inplace) / 6.


# Squeeze-And-ExciteÄ£¿é
class SEModule(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.se = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y=self.avg_pool(x).view(b, c)
        y=self.se(y)
        y = Hsigmoid(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class Bottleneck(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,exp_channels,stride,se='True',nl='HS'):
        super(Bottleneck, self).__init__()
        padding = (kernel_size - 1) // 2
        if nl == 'RE':
            self.nlin_layer = F.relu6
        elif nl == 'HS':
            self.nlin_layer = Hswish
        self.stride=stride
        if se:
            self.se=SEModule(exp_channels)
        else:
            self.se=None
        self.conv1=nn.Conv2d(in_channels,exp_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn1 = nn.BatchNorm2d(exp_channels)
        self.conv2=nn.Conv2d(exp_channels,exp_channels,kernel_size=kernel_size,stride=stride,
                             padding=padding,groups=exp_channels,bias=False)
        self.bn2=nn.BatchNorm2d(exp_channels)
        self.conv3=nn.Conv2d(exp_channels,out_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn3=nn.BatchNorm2d(out_channels)
        # ÏÈ³õÊ¼»¯Ò»¸ö¿ÕÐòÁÐ£¬Ö®ºó¸ÄÔìÆä³ÉÎª²Ð²îÁ´½Ó
        self.shortcut = nn.Sequential()
        # Ö»ÓÐ²½³¤Îª1ÇÒÊäÈëÊä³öÍ¨µÀ²»ÏàÍ¬Ê±²Å²ÉÓÃÌøÔ¾Á¬½Ó(ÏëÒ»ÏÂÌøÔ¾Á´½ÓµÄ¹ý³Ì£¬ÊäÈëÊä³öÍ¨µÀÏàÍ¬Õâ¸öÌøÔ¾Á¬½Ó¾ÍÃ»ÒâÒåÁË)
        if stride == 1 and in_channels != out_channels:
            self.shortcut = nn.Sequential(
                # ÏÂÃæµÄ²Ù×÷¾í»ý²»¸Ä±ä³ß´çºÍÍ¨µÀÊý
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self,x):
        out=self.nlin_layer(self.bn1(self.conv1(x)))
        if self.se is not None:
            out=self.bn2(self.conv2(out))
            out=self.nlin_layer(self.se(out))
        else:
            out = self.nlin_layer(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride == 1 else out
        return out


class MobileNetV3_large(nn.Module):
    # (out_channels,kernel_size,exp_channels,stride,se,nl)
    cfg=[
        (16,3,16,1,False,'RE'),
        (24,3,64,2,False,'RE'),
        (24,3,72,1,False,'RE'),
        (40,5,72,2,True,'RE'),
        (40,5,120,1,True,'RE'),
        (40,5,120,1,True,'RE'),
        (80,3,240,2,False,'HS'),
        (80,3,200,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (112,3,480,1,True,'HS'),
        (112,3,672,1,True,'HS'),
        (160,5,672,2,True,'HS'),
        (160,5,960,1,True,'HS'),
        (160,5,960,1,True,'HS')
    ]
    def __init__(self,num_classes=5):
        super(MobileNetV3_large,self).__init__()
        self.conv1=nn.Conv2d(3,16,3,2,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(16)
        # ¸ù¾ÝcfgÊý×é×Ô¶¯Éú³ÉËùÓÐµÄBottleneck²ã
        self.layers = self._make_layers(in_channels=16)
        self.conv2=nn.Conv2d(160,960,1,stride=1,bias=False)
        self.bn2=nn.BatchNorm2d(960)
        # ¾í»ýºó²»¸úBN£¬¾ÍÓ¦¸Ã°ÑbiasÉèÖÃÎªTrue
        self.conv3=nn.Conv2d(960,1280,1,1,padding=0,bias=True)
        self.conv4=nn.Conv2d(1280,num_classes,1,stride=1,padding=0,bias=True)

    def _make_layers(self,in_channels):
        layers=[]
        for out_channels,kernel_size,exp_channels,stride,se,nl in self.cfg:
            layers.append(
                Bottleneck(in_channels,out_channels,kernel_size,exp_channels,stride,se,nl)
            )
            in_channels=out_channels
        return nn.Sequential(*layers)

    def forward(self,x):
        out=Hswish(self.bn1(self.conv1(x)))
        out=self.layers(out)
        out=Hswish(self.bn2(self.conv2(out)))
        out=F.avg_pool2d(out,7)
        out=Hswish(self.conv3(out))
        out=self.conv4(out)
        # ÒòÎªÔ­ÂÛÎÄÖÐ×îºóÒ»²ãÊÇ¾í»ý²ãÀ´ÊµÏÖÈ«Á¬½ÓµÄÐ§¹û£¬Î¬¶ÈÊÇËÄÎ¬µÄ£¬ºóÁ½Î¬ÊÇ1£¬ÔÚ¼ÆËãËðÊ§º¯ÊýµÄÊ±ºòÒªÇó¶þÎ¬£¬Òò´ËÔÚÕâÀïÐèÒª×öÒ»¸öresize
        a,b=out.size(0),out.size(1)
        out=out.view(a,b)
        return out

In [8]:
model=MobileNetV3_large()

In [9]:
# x=torch.randn(2,3,224,224)
# y=model(x)
# print(y.size())
# print(y)

In [10]:
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader

In [11]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset

random.seed(1)

class flowerDataset(Dataset):
    # 自定义Dataset类，必须继承Dataset并重写__init__和__getitem__函数
    def __init__(self, data_dir, transform=None):
        """
            花朵分类任务的Dataset
            :param data_dir: str, 数据集所在路径
            :param transform: torch.transform，数据预处理，默认不进行预处理
        """
        # data_info存储所有图片路径和标签（元组的列表），在DataLoader中通过index读取样本
        self.data_info = self.get_img_info(data_dir)
        self.transform = transform

    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        # 打开图片，默认为PIL，需要转成RGB
        img = Image.open(path_img).convert('RGB')
        # 如果预处理的条件不为空，应该进行预处理操作
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data_info)

    # 自定义方法，用于返回所有图片的路径以及标签
    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                # listdir为列出文件夹下所有文件和文件夹名
                img_names = os.listdir(os.path.join(root, sub_dir))
                # 过滤出所有后缀名为jpg的文件名（那当然也就把文件夹过滤掉了）
#                 img_names = list(filter(lambda x: x.endswith('.jpeg'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    # 在该任务中，文件夹名等于标签名
                    label = sub_dir
                    data_info.append((path_img, int(label)))
        return data_info

In [12]:
#宏定义一些数据，如epoch数，batchsize等
MAX_EPOCH=45
BATCH_SIZE=128
LR=0.0001
log_interval=30
val_interval=1

In [13]:
# ============================ step 1/5 数据 ============================
# split_dir=os.path.join("MobileNetV3-master",".","data","splitData")
split_dir=r'/home/fanrz/Desktop/torchmob/'
train_dir=r'/home/fanrz/Desktop/testlarge/aptos/tra2/'
valid_dir=os.path.join(split_dir,"val")

In [14]:
#对训练集所需要做的预处理
train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [15]:
#对验证集所需要做的预处理
valid_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [16]:
# 构建MyDataset实例
train_data=flowerDataset(data_dir=train_dir,transform=train_transform)
valid_data=flowerDataset(data_dir=valid_dir,transform=valid_transform)

In [17]:
print(len(train_data.data_info))

5035


In [18]:
# 构建DataLoader
# 训练集数据最好打乱
# DataLoader的实质就是把数据集加上一个索引号，再返回
train_loader=DataLoader(dataset=train_data,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True)
valid_loader=DataLoader(dataset=valid_data,batch_size=BATCH_SIZE,drop_last=True)

In [19]:
# ============================ step 2/5 模型 ============================
if torch.cuda.is_available():
    model=nn.DataParallel(model)
    model.cuda()

In [20]:
# ============================ step 3/5 损失函数 ============================
criterion=nn.CrossEntropyLoss()
# ============================ step 4/5 优化器 ============================
optimizer=optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.99))# 选择优化器

In [21]:
# ============================ step 5/5 训练 ============================
# 记录每一次的数据，方便绘图
train_curve=list()
valid_curve=list()
model.train()
accurancy_global=0.0
for epoch in range(MAX_EPOCH):
    loss_mean=0.
    correct=0.
    total=0.
    running_loss = 0.0

    for i,data in enumerate(train_loader):
        img,label=data
        img = Variable(img)
        label = Variable(label)
        if torch.cuda.is_available():
            img=img.cuda()
            label=label.cuda()
        
#         print(img.size())
        # 前向传播
        out=model(img)
        optimizer.zero_grad()  # 归0梯度
        loss=criterion(out,label)#得到损失函数

        print_loss=loss.data.item()

        loss.backward()#反向传播
        optimizer.step()#优化
        if (i+1)%log_interval==0:
            print('epoch:{},loss:{:.4f}'.format(epoch+1,loss.data.item()))
        _, predicted = torch.max(out.data, 1)
        total += label.size(0)
        if (i+1)%log_interval==0:
            print("============================================")
            print("源数据标签：",label)
            print("============================================")
            print("预测结果：",predicted)
            print("相等的结果为：",predicted == label)
        correct += (predicted == label).sum()
        if (i+1)%log_interval==0:
            print(correct.item() / total)
#         print(correct.item())
    print("============================================")
    accurancy=correct.item() / total
    if accurancy>accurancy_global:
        torch.save(model.state_dict(), './weights/best326.pkl')
        print("准确率由：", accurancy_global, "上升至：", accurancy, "已更新并保存权值为weights/best.pkl")
        accurancy_global=accurancy
    print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, 100*accurancy))
# torch.save(model.state_dict(), './weights/last607.pkl')
print("训练完毕，权重已保存为：weights/last.pkl")


epoch:1,loss:1.3339
源数据标签： tensor([3, 3, 2, 0, 1, 1, 4, 2, 0, 2, 0, 4, 0, 0, 3, 4, 2, 2, 0, 2, 3, 0, 2, 1,
        2, 2, 1, 3, 0, 1, 1, 4, 1, 1, 2, 2, 4, 3, 2, 2, 4, 1, 1, 4, 2, 3, 1, 1,
        2, 3, 1, 4, 4, 4, 3, 3, 3, 0, 2, 1, 4, 4, 0, 4, 1, 0, 1, 4, 2, 3, 0, 1,
        0, 2, 3, 3, 1, 2, 4, 0, 1, 1, 3, 0, 4, 2, 4, 4, 1, 2, 1, 4, 0, 1, 4, 4,
        1, 2, 2, 1, 4, 0, 1, 1, 1, 4, 4, 4, 2, 1, 1, 3, 1, 2, 1, 4, 3, 3, 4, 2,
        0, 3, 4, 2, 2, 2, 3, 2], device='cuda:0')
预测结果： tensor([1, 3, 1, 0, 0, 2, 3, 2, 0, 2, 3, 1, 0, 0, 1, 3, 0, 2, 0, 2, 3, 0, 2, 3,
        2, 4, 3, 1, 0, 3, 1, 3, 3, 3, 0, 2, 3, 3, 3, 1, 3, 3, 3, 1, 2, 1, 3, 2,
        2, 1, 3, 1, 0, 1, 3, 3, 3, 0, 2, 2, 1, 2, 0, 3, 2, 0, 3, 3, 3, 3, 0, 2,
        0, 1, 3, 3, 2, 3, 2, 0, 1, 3, 3, 0, 2, 3, 2, 3, 3, 2, 1, 3, 0, 3, 3, 2,
        3, 2, 2, 1, 2, 1, 2, 2, 3, 2, 3, 2, 2, 2, 0, 3, 3, 2, 3, 3, 2, 3, 3, 2,
        0, 3, 3, 2, 2, 2, 2, 2], device='cuda:0')
相等的结果为： tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 

epoch:6,loss:0.7636
源数据标签： tensor([1, 1, 0, 0, 3, 4, 0, 3, 4, 0, 0, 4, 0, 0, 0, 1, 1, 2, 3, 0, 1, 1, 1, 4,
        4, 4, 0, 0, 2, 4, 4, 1, 4, 0, 3, 4, 2, 3, 1, 3, 2, 4, 3, 1, 4, 4, 2, 0,
        4, 4, 1, 2, 0, 3, 1, 3, 2, 3, 2, 2, 2, 3, 2, 4, 1, 3, 2, 0, 4, 1, 3, 4,
        4, 1, 0, 3, 2, 0, 3, 0, 0, 4, 1, 1, 1, 0, 0, 0, 3, 0, 1, 2, 4, 4, 2, 1,
        2, 3, 1, 0, 2, 1, 2, 2, 2, 4, 3, 4, 3, 2, 1, 1, 2, 3, 2, 2, 4, 2, 1, 2,
        1, 1, 2, 2, 0, 1, 4, 1], device='cuda:0')
预测结果： tensor([1, 2, 0, 0, 3, 4, 0, 3, 1, 0, 0, 3, 0, 0, 0, 4, 4, 1, 3, 0, 1, 1, 3, 4,
        2, 2, 0, 0, 2, 3, 1, 1, 1, 0, 3, 4, 2, 4, 1, 4, 2, 1, 1, 1, 1, 1, 2, 0,
        1, 2, 1, 2, 0, 3, 1, 4, 2, 3, 2, 2, 2, 4, 2, 3, 3, 2, 2, 2, 3, 1, 4, 4,
        4, 1, 0, 3, 2, 0, 3, 0, 0, 4, 1, 1, 1, 0, 0, 0, 3, 0, 1, 2, 4, 3, 2, 3,
        2, 3, 1, 0, 2, 0, 2, 2, 2, 1, 3, 4, 3, 2, 1, 3, 2, 2, 2, 2, 4, 2, 4, 2,
        1, 3, 2, 2, 0, 1, 1, 1], device='cuda:0')
相等的结果为： tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 

epoch:11,loss:0.5265
源数据标签： tensor([0, 4, 2, 4, 3, 3, 4, 1, 0, 2, 2, 0, 0, 4, 3, 3, 1, 3, 2, 1, 3, 1, 4, 0,
        1, 2, 1, 2, 4, 0, 2, 1, 2, 3, 3, 4, 2, 4, 4, 1, 4, 3, 2, 0, 3, 3, 4, 4,
        1, 1, 3, 2, 3, 0, 3, 1, 1, 2, 1, 4, 1, 4, 2, 3, 2, 1, 0, 2, 0, 3, 2, 1,
        1, 2, 1, 4, 3, 0, 0, 3, 2, 0, 3, 4, 1, 1, 3, 2, 1, 4, 0, 4, 4, 0, 3, 3,
        3, 2, 4, 4, 1, 2, 0, 0, 0, 1, 2, 1, 4, 3, 3, 0, 1, 3, 0, 2, 2, 0, 3, 0,
        3, 2, 2, 1, 2, 4, 2, 0], device='cuda:0')
预测结果： tensor([2, 4, 2, 3, 3, 3, 3, 1, 0, 3, 3, 0, 0, 3, 3, 2, 1, 2, 2, 1, 3, 1, 4, 0,
        1, 2, 1, 2, 4, 0, 2, 1, 2, 3, 3, 1, 2, 4, 4, 1, 3, 3, 2, 0, 3, 3, 1, 4,
        3, 1, 2, 2, 2, 0, 3, 1, 1, 2, 1, 2, 1, 4, 2, 3, 2, 1, 0, 2, 0, 3, 2, 1,
        1, 2, 1, 3, 1, 0, 0, 3, 2, 0, 3, 1, 1, 3, 3, 2, 1, 4, 0, 4, 4, 0, 3, 3,
        3, 2, 4, 3, 1, 2, 0, 0, 0, 2, 2, 1, 1, 3, 3, 0, 4, 1, 0, 2, 2, 0, 3, 0,
        3, 2, 2, 1, 3, 3, 2, 0], device='cuda:0')
相等的结果为： tensor([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,

epoch:16,loss:0.2822
源数据标签： tensor([2, 0, 4, 0, 3, 4, 0, 4, 1, 2, 0, 1, 2, 2, 4, 0, 1, 4, 3, 3, 4, 3, 3, 2,
        1, 2, 3, 1, 3, 0, 1, 4, 3, 1, 4, 3, 4, 4, 2, 2, 3, 0, 2, 3, 1, 1, 1, 1,
        1, 1, 3, 3, 1, 0, 3, 1, 2, 0, 1, 2, 2, 2, 1, 2, 0, 4, 0, 4, 2, 3, 3, 1,
        1, 1, 2, 3, 0, 2, 2, 3, 3, 1, 4, 1, 3, 3, 0, 4, 4, 0, 1, 1, 3, 3, 1, 3,
        1, 2, 2, 4, 2, 2, 2, 3, 1, 3, 3, 2, 0, 2, 1, 1, 3, 2, 0, 4, 3, 1, 1, 2,
        4, 1, 0, 3, 1, 4, 4, 1], device='cuda:0')
预测结果： tensor([2, 0, 4, 0, 3, 4, 0, 4, 1, 2, 0, 1, 2, 2, 4, 0, 1, 4, 4, 3, 3, 1, 3, 2,
        1, 2, 3, 1, 4, 0, 4, 4, 3, 1, 4, 3, 4, 4, 2, 2, 3, 0, 2, 1, 1, 3, 3, 1,
        1, 1, 4, 3, 1, 0, 3, 1, 2, 0, 1, 2, 2, 2, 1, 2, 0, 2, 0, 4, 2, 3, 3, 1,
        1, 1, 2, 3, 0, 2, 2, 3, 3, 1, 4, 1, 4, 1, 0, 4, 4, 0, 2, 3, 3, 4, 3, 3,
        1, 2, 2, 4, 2, 2, 2, 3, 1, 3, 3, 2, 0, 2, 1, 1, 3, 2, 0, 4, 3, 1, 1, 2,
        4, 2, 0, 3, 1, 4, 4, 1], device='cuda:0')
相等的结果为： tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

epoch:21,loss:0.5960
源数据标签： tensor([4, 1, 3, 3, 4, 0, 0, 3, 2, 4, 0, 3, 4, 1, 4, 0, 3, 1, 1, 3, 0, 0, 4, 4,
        4, 1, 0, 4, 4, 1, 1, 3, 0, 3, 4, 3, 2, 3, 4, 1, 1, 4, 4, 4, 4, 4, 2, 3,
        4, 4, 4, 2, 4, 2, 0, 1, 3, 1, 0, 0, 1, 1, 1, 4, 0, 2, 2, 2, 3, 0, 2, 2,
        4, 2, 0, 2, 1, 1, 1, 1, 1, 3, 3, 2, 3, 3, 3, 0, 2, 0, 4, 4, 2, 1, 1, 1,
        1, 0, 3, 0, 3, 2, 2, 0, 2, 3, 2, 4, 0, 0, 1, 2, 1, 3, 0, 3, 3, 0, 3, 4,
        4, 2, 4, 1, 1, 2, 2, 3], device='cuda:0')
预测结果： tensor([4, 1, 3, 1, 3, 0, 0, 3, 2, 4, 0, 3, 4, 1, 4, 0, 3, 1, 3, 3, 0, 0, 4, 3,
        4, 1, 0, 4, 3, 1, 1, 3, 0, 3, 2, 3, 2, 3, 4, 1, 2, 4, 4, 4, 4, 4, 2, 3,
        4, 4, 4, 2, 1, 2, 0, 1, 3, 1, 0, 0, 1, 1, 1, 4, 0, 2, 4, 2, 3, 0, 2, 2,
        3, 2, 0, 2, 1, 3, 1, 2, 1, 3, 4, 2, 4, 4, 1, 0, 2, 0, 2, 4, 2, 1, 1, 2,
        1, 0, 3, 0, 1, 1, 4, 0, 2, 3, 4, 4, 2, 0, 1, 2, 1, 3, 0, 3, 3, 0, 3, 1,
        3, 1, 4, 4, 4, 2, 2, 3], device='cuda:0')
相等的结果为： tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

epoch:26,loss:0.1338
源数据标签： tensor([3, 2, 2, 3, 3, 4, 2, 2, 4, 2, 0, 2, 2, 3, 0, 1, 0, 3, 3, 4, 1, 2, 0, 1,
        2, 4, 1, 3, 0, 1, 1, 3, 1, 4, 2, 3, 2, 1, 1, 2, 0, 1, 3, 4, 1, 1, 1, 2,
        2, 0, 3, 4, 4, 3, 0, 0, 0, 0, 4, 0, 4, 4, 0, 1, 2, 1, 0, 0, 0, 1, 1, 0,
        3, 3, 2, 2, 2, 2, 0, 1, 3, 3, 1, 1, 1, 2, 2, 4, 0, 0, 3, 2, 0, 1, 3, 1,
        4, 2, 0, 2, 0, 4, 4, 1, 0, 3, 1, 4, 0, 0, 4, 0, 2, 4, 1, 3, 0, 3, 2, 2,
        1, 3, 0, 2, 1, 3, 2, 0], device='cuda:0')
预测结果： tensor([3, 2, 2, 3, 3, 4, 2, 2, 4, 2, 0, 2, 2, 3, 0, 1, 0, 3, 3, 4, 1, 2, 0, 1,
        2, 4, 1, 3, 0, 1, 1, 3, 1, 4, 2, 3, 2, 1, 1, 2, 0, 1, 3, 4, 1, 1, 1, 2,
        2, 0, 3, 4, 4, 3, 0, 0, 0, 0, 4, 0, 4, 4, 0, 1, 2, 1, 0, 0, 0, 4, 1, 0,
        3, 3, 2, 2, 2, 1, 0, 1, 1, 3, 1, 1, 1, 2, 2, 4, 0, 0, 3, 2, 0, 1, 3, 1,
        4, 2, 0, 2, 0, 4, 4, 1, 0, 3, 1, 4, 0, 0, 4, 0, 2, 4, 1, 3, 0, 3, 2, 2,
        1, 3, 0, 2, 1, 2, 2, 0], device='cuda:0')
相等的结果为： tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

准确率由： 0.960136217948718 上升至： 0.9619391025641025 已更新并保存权值为weights/best.pkl
第31个epoch的识别准确率为：96%
epoch:32,loss:0.1477
源数据标签： tensor([3, 1, 1, 1, 3, 3, 1, 0, 1, 2, 4, 0, 1, 1, 1, 4, 1, 2, 2, 0, 1, 1, 1, 4,
        4, 1, 1, 1, 0, 1, 1, 0, 4, 1, 4, 1, 0, 0, 1, 2, 3, 3, 3, 2, 1, 3, 4, 3,
        0, 1, 1, 3, 1, 3, 0, 0, 3, 4, 1, 1, 2, 1, 2, 3, 3, 1, 2, 2, 2, 3, 3, 2,
        3, 3, 1, 3, 3, 0, 3, 1, 1, 1, 4, 1, 1, 0, 0, 1, 3, 4, 1, 0, 3, 4, 1, 4,
        0, 0, 4, 1, 0, 1, 2, 3, 4, 0, 4, 2, 1, 0, 0, 4, 3, 1, 1, 0, 1, 0, 1, 3,
        4, 3, 4, 0, 3, 0, 1, 2], device='cuda:0')
预测结果： tensor([3, 1, 1, 1, 3, 3, 4, 0, 1, 2, 4, 0, 1, 1, 1, 4, 1, 2, 2, 0, 1, 1, 1, 4,
        4, 1, 1, 1, 0, 1, 1, 0, 4, 1, 4, 1, 0, 0, 1, 2, 3, 3, 3, 2, 1, 3, 4, 3,
        0, 1, 1, 3, 1, 3, 0, 0, 3, 4, 1, 1, 2, 1, 2, 3, 3, 1, 2, 2, 2, 3, 3, 2,
        3, 3, 3, 3, 1, 0, 3, 1, 1, 2, 4, 1, 1, 0, 0, 3, 3, 4, 1, 0, 3, 4, 1, 4,
        0, 0, 4, 1, 0, 1, 2, 3, 4, 0, 4, 2, 1, 0, 0, 4, 3, 1, 1, 0, 1, 0, 1, 3,
        4, 3, 4, 0, 3

准确率由： 0.9685496794871795 上升至： 0.9713541666666666 已更新并保存权值为weights/best.pkl
第36个epoch的识别准确率为：97%
epoch:37,loss:0.1005
源数据标签： tensor([2, 1, 2, 0, 0, 1, 3, 3, 4, 1, 2, 0, 4, 3, 0, 1, 2, 2, 0, 4, 2, 0, 1, 2,
        0, 2, 0, 3, 1, 2, 0, 0, 1, 2, 2, 2, 4, 3, 3, 0, 1, 0, 0, 4, 0, 1, 3, 4,
        4, 2, 3, 4, 3, 3, 0, 4, 1, 3, 4, 0, 0, 3, 2, 3, 4, 0, 0, 2, 3, 1, 1, 2,
        1, 4, 4, 1, 0, 3, 0, 1, 1, 2, 1, 2, 0, 3, 3, 4, 3, 3, 1, 0, 3, 4, 4, 0,
        2, 4, 2, 0, 0, 0, 2, 1, 2, 2, 2, 4, 1, 1, 1, 0, 0, 1, 1, 1, 4, 2, 0, 0,
        2, 2, 3, 3, 4, 4, 1, 0], device='cuda:0')
预测结果： tensor([2, 1, 2, 0, 0, 1, 3, 3, 4, 1, 2, 0, 4, 3, 0, 1, 2, 2, 0, 4, 4, 1, 1, 2,
        0, 2, 0, 3, 1, 2, 0, 0, 1, 2, 2, 2, 4, 3, 3, 0, 1, 0, 0, 4, 0, 1, 3, 4,
        4, 2, 3, 4, 3, 3, 0, 4, 1, 1, 4, 0, 0, 3, 2, 3, 4, 0, 0, 2, 3, 1, 1, 2,
        1, 4, 4, 1, 0, 3, 0, 1, 1, 2, 1, 4, 0, 3, 3, 4, 3, 3, 1, 0, 3, 4, 4, 0,
        2, 4, 2, 0, 0, 0, 2, 1, 2, 2, 2, 4, 1, 1, 1, 0, 0, 1, 1, 1, 4, 2, 0, 0,
        2, 2, 3, 3, 

epoch:42,loss:0.0565
源数据标签： tensor([2, 0, 3, 3, 2, 1, 4, 1, 4, 1, 3, 4, 4, 4, 4, 3, 0, 4, 4, 3, 2, 1, 4, 1,
        3, 1, 4, 2, 0, 3, 0, 1, 1, 0, 3, 4, 4, 4, 1, 1, 0, 2, 2, 3, 2, 2, 0, 0,
        1, 0, 1, 4, 1, 1, 0, 2, 3, 4, 3, 2, 1, 4, 1, 4, 2, 3, 1, 1, 3, 4, 1, 2,
        1, 0, 4, 3, 0, 3, 1, 2, 1, 3, 1, 1, 4, 1, 1, 4, 1, 0, 2, 2, 0, 0, 0, 0,
        0, 0, 3, 0, 2, 1, 3, 0, 2, 0, 3, 3, 2, 0, 1, 0, 1, 3, 1, 4, 2, 3, 3, 2,
        3, 0, 2, 3, 0, 0, 3, 3], device='cuda:0')
预测结果： tensor([2, 0, 3, 3, 2, 1, 4, 1, 4, 1, 3, 4, 4, 4, 4, 3, 0, 4, 4, 3, 2, 1, 4, 1,
        3, 1, 4, 2, 0, 3, 0, 1, 1, 0, 3, 4, 4, 4, 1, 1, 0, 2, 2, 3, 2, 2, 0, 0,
        1, 0, 3, 4, 1, 1, 0, 2, 3, 4, 3, 2, 1, 4, 1, 4, 2, 3, 1, 1, 3, 4, 1, 2,
        1, 0, 4, 3, 0, 3, 1, 2, 1, 3, 1, 1, 4, 1, 1, 4, 1, 0, 2, 2, 0, 0, 0, 0,
        0, 0, 3, 0, 2, 1, 3, 0, 2, 0, 3, 3, 2, 0, 1, 0, 1, 3, 4, 4, 2, 3, 3, 2,
        3, 0, 2, 3, 0, 0, 3, 4], device='cuda:0')
相等的结果为： tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [22]:
print('have done')

have done
